# De XML para CSV

O formato XML não facilita muito a vida de quem quer utilizar os dados de forma mais simples. Após descompactar os [Acórdãos do Segundo Grau](https://osf.io/as8uv/), o pesquisador fica com vários XMLs no diretório, totalizando cerca de 2.2G de dados. Com esse volume, já sentimos uma certa lentidão para realizar a carga dos arquivos, imagine fazer operações de agrupamento, ordenação, etc. Ou seja, temos muito material para 'brincar'!

Considerando que o formato CSV acabou tornando-se muito popular para disponibilizar dados na Web, então, vamos fazer a conversão! 

Em virtude do volume, poderíamos utilizar o [Dask](https://dask.org/), por exemplo, para paralelizar o parser dos arquivos, mas a ideia é começarmos simples e, em outros notebooks, explorarmos outros tópicos/tecnologias. 

O XML possui algumas colunas que não agregam valor para análise, haja vista que são dados utilizados internamente pelo motor de buscas do TJSE (o campo Chave, por exemplo). Sendo assim, utilizamos somente as colunas:
 * txtAcordao: Texto do acórdão;
 * data: Data de publicação do acórdão;
 * txtVoto: Texto relatando o voto de cada Desembargador;
 * NroProcesso: Número do processo; 
 * NroAcordao: Número do acórdão;
 * DescRecurso: Tipo de recurso;
 * DesRelator: Nome do relator do acórdão;
 * OrgaoJulgador: Órgão que julgou o acórdão;
 * txtEmenta: Texto da ementa. Este é um dos campos textuais mais importantes, pois representa um resumo do que foi tratado pelo acórdão; 
 * txtRelatorio: Texto do relatório do acórdão.
 
 
Para entender um pouco mais sobre o que é um acórdão recomendo a leitura [desse](https://www.infoescola.com/direito/acordao/) artigo, ou do Cap. 2 de [minha dissertação](http://osf.io/x3rbn/?action=download) se tiver com um pouco mais de tempo. :)
 

In [1]:
#Caso os XMLs não coubessem na memória, seria melhor ir gerando o CSV conforme o XML fosse lido.
#Como não é o caso aqui, carregaremos simplesmente os registros na memória.

from pathlib import Path
import xml.etree.ElementTree as ET
import numpy as np

acordaos_dict = {'txtAcordao':[], 
      'data':[], 
      'txtVoto':[], 
      'NroProcesso':[], 
      'NroAcordao':[], 
      'DescRecurso':[], 
      'DesRelator':[], 
      'OrgaoJulgador':[], 
      'txtEmenta':[], 
      'txtRelatorio':[]}

def popular_dicionario(arquivo, dicionario):            
    tree = ET.parse(arquivo)
    root = tree.getroot()
    for doc in tree.findall(".//doc"):
        for k in dicionario.keys():
            try:
                texto = doc.find(f"./field[@name='{k}']").text
                dicionario[k].append(texto)
            except:
                dicionario[k].append(np.nan)

xml_dir = Path("dataset/asg/")

for xml in xml_dir.glob('*.xml'):
    popular_dicionario(xml, acordaos_dict)

## Limpeza de Dados

Além de fazer a conversão, que tal aproveitarmos os dados em memória para executar alguns tratamentos nos dados? Para tal, utilizaremos o [Pandas](https://pandas.pydata.org/).

In [2]:
import pandas as pd

df = pd.DataFrame(acordaos_dict)
del acordaos_dict #Não precisamos mais do dicionário ocupando nossa preciosa memória
df.head()

,DesRelator,DescRecurso,NroAcordao,NroProcesso,OrgaoJulgador,data,txtAcordao,txtEmenta,txtRelatorio,txtVoto
0,Des. Cezário Siqueira Neto,Apelação,20165410,200200205551,2ª CÂMARA CÍVEL,2016-04-14T00:00:00Z,"Vistos, relatados e discutidos estes autos, A...",APELAÇÃO CÍVEL. AÇÃO REVISIONAL. SUCESSIVOS C...,Desembargador Cezário Siqueira Neto (Relator)...,Desembargador Cezário Siqueira Neto (Relator)...
1,Des. Luiz Antônio Araújo Mendonça,Embargos de Declaração,201420012,200300104705,TRIBUNAL PLENO,2014-12-09T00:00:00Z,"Vistos, relatados e discutidos estes autos em...",Embargos de Declaração. Alegação de omissão. ...,Desembargador Luiz Antônio Araújo Mendonça (R...,Desembargador Luiz Antônio Araújo Mendonça (R...
2,Des. Roberto Eugenio da Fonseca Porto,Apelação,20149458,200300206387,1ª CÂMARA CÍVEL,2014-07-03T00:00:00Z,"Vistos, relatados e discutidos os presentes a...",APELAÇÃO CÍVEL ? CONTRATO BANCÁRIO ? NOVO JUL...,Des. ROBERTO EUGENIO DA FONSECA PORTO (Relato...,Des. ROBERTO EUGENIO DA FONSECA PORTO (Relato...
3,Dra. Ana Bernadete Leite de Carvalho Andrade,Apelação,20161032,200400205720,1ª CÂMARA CÍVEL,2016-02-11T00:00:00Z,"Vistos, relatados e discutidos estes autos, a...",RECURSO EXTRAORDINÁRIO SOBRESTADO NA PRESIDÊN...,JUÍZA CONVOCADA ANA BERNADETE LEITE DE CARVAL...,JUÍZA CONVOCADA ANA BERNADETE LEITE DE CARVAL...
4,Des. Osório de Araújo Ramos Filho,Apelação,201511058,200400210132,1ª CÂMARA CÍVEL,2015-07-17T00:00:00Z,"Vistos, relatados e discutidos os autos da pr...",APELAÇÃO CÍVEL ? AÇÃO REVISIONAL ? CONTRATO B...,"O BANCO DO ESTADO DE SERGIPE S/A - BANESE, in...","O BANCO DO ESTADO DE SERGIPE S/A - BANESE, in..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181994 entries, 0 to 181993
Data columns (total 10 columns):
DesRelator       181720 non-null object
DescRecurso      181993 non-null object
NroAcordao       181994 non-null object
NroProcesso      181994 non-null object
OrgaoJulgador    181994 non-null object
data             181994 non-null object
txtAcordao       181935 non-null object
txtEmenta        181994 non-null object
txtRelatorio     172528 non-null object
txtVoto          181994 non-null object
dtypes: object(10)
memory usage: 13.9+ MB


Beleza! Temos um dataframe com 181994 acórdãos. Vamos começar dando uma olhada nos nomes dos relatores. Será que tem nomes iguais com *cases* diferentes?

In [4]:
display(df.DesRelator.unique().size - df.DesRelator.str.lower().unique().size)

11

Acertamos no palpite! Vamos converter todos os nomes para *lower case* e ver o que acontece.

In [5]:
df.DesRelator = df.DesRelator.str.lower()

df.DesRelator.sort_values().unique()

array(['corregedor geral de justiça', 'des. alberto romeu gouveia leite',
       'des. aloiso de abreu lima', 'des. artur oscar de oliveira deda',
       'des. cezário siqueira neto', 'des. cláudio dinart déda chagas',
       'des. diógenes barreto', 'des. edson ulisses de melo',
       'des. epaminondas s. de andrade lima',
       'des. fernando ribeiro franco', 'des. gilson gois soares',
       'des. jose antonio de a. goes', 'des. jose barreto prado',
       'des. jose nolasco de carvalho', 'des. josé alves neto',
       'des. josé artêmio barreto', 'des. josé dos anjos',
       'des. luiz antônio araújo mendonça', 'des. manoel cândido filho',
       'des. manuel pascoal nabuco d`avila',
       'des. netônio bezerra machado', 'des. osório de araújo ramos filho',
       'des. ricardo múcio santana de a. lima',
       'des. ricardo múcio santana de abreu lima',
       'des. roberto eugenio da fonseca porto',
       'des. ruy pinheiro da silva',
       'desa. ana lúcia freire de a. dos

Os nomes 'des. ricardo múcio santana de a. lima' e 'des. ricardo múcio santana de abreu lima' representam a mesma pessoa. Para economizar alguns *keystrokes*, manteremos somente a versão abreviada:

In [6]:
df.DesRelator = df.DesRelator.replace('des. ricardo múcio santana de abreu lima', 'des. ricardo múcio santana de a. lima')

Aparentemente, os nomes estão ok. Para evitar situações desse tipo, vamos converter `DescRecurso` e `OrgaoJulgador` para caixa-baixa.

In [7]:
df.DescRecurso = df.DescRecurso.str.lower()
display(len(df.DescRecurso.unique()))
display(df.DescRecurso.drop_duplicates().head()) #visualizando somente os primeiros tipos de recurso

df.OrgaoJulgador = df.OrgaoJulgador.str.lower()
df.OrgaoJulgador.unique() #aqui são poucos, podemos visualizar todos


141

0                       apelação
1         embargos de declaração
55    recurso em sentido estrito
57               ação rescisória
79          mandado de segurança
Name: DescRecurso, dtype: object

array(['2ª câmara cível', 'tribunal pleno', '1ª câmara cível',
       'câmara criminal', 'câmaras cíveis reunidas',
       'conselho da magistratura', 'câmara cível'], dtype=object)

Massa! Vamos olhar agora o campo `data`. Já que nenhuma data está com o valor nulo, vamos checar se todas estão no formato correto.

In [8]:
idx_formato_correto = df.data.str.match(r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z')
len(df) - sum(idx_formato_correto)

0

Ótimo! Todas as datas estão com formato válido. Vamos ver se o Pandas consegue convertê-las:

In [9]:
df.data = pd.to_datetime(df.data)

O CSV não armazena informações de tipo (o [Parquet](https://parquet.apache.org/) está ganhando popularidade e esse é um dos motivos), mas, pelo menos, já sabemos que o Pandas está conseguindo converter sem problemas. Vamos dar mais uma olhada no nosso dataframe:

In [10]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181994 entries, 0 to 181993
Data columns (total 10 columns):
DesRelator       181720 non-null object
DescRecurso      181993 non-null object
NroAcordao       181994 non-null object
NroProcesso      181994 non-null object
OrgaoJulgador    181994 non-null object
data             181994 non-null datetime64[ns]
txtAcordao       181935 non-null object
txtEmenta        181994 non-null object
txtRelatorio     172528 non-null object
txtVoto          181994 non-null object
dtypes: datetime64[ns](1), object(9)
memory usage: 13.9+ MB


,DesRelator,DescRecurso,NroAcordao,NroProcesso,OrgaoJulgador,data,txtAcordao,txtEmenta,txtRelatorio,txtVoto
0,des. cezário siqueira neto,apelação,20165410,200200205551,2ª câmara cível,2016-04-14,"Vistos, relatados e discutidos estes autos, A...",APELAÇÃO CÍVEL. AÇÃO REVISIONAL. SUCESSIVOS C...,Desembargador Cezário Siqueira Neto (Relator)...,Desembargador Cezário Siqueira Neto (Relator)...
1,des. luiz antônio araújo mendonça,embargos de declaração,201420012,200300104705,tribunal pleno,2014-12-09,"Vistos, relatados e discutidos estes autos em...",Embargos de Declaração. Alegação de omissão. ...,Desembargador Luiz Antônio Araújo Mendonça (R...,Desembargador Luiz Antônio Araújo Mendonça (R...
2,des. roberto eugenio da fonseca porto,apelação,20149458,200300206387,1ª câmara cível,2014-07-03,"Vistos, relatados e discutidos os presentes a...",APELAÇÃO CÍVEL ? CONTRATO BANCÁRIO ? NOVO JUL...,Des. ROBERTO EUGENIO DA FONSECA PORTO (Relato...,Des. ROBERTO EUGENIO DA FONSECA PORTO (Relato...
3,dra. ana bernadete leite de carvalho andrade,apelação,20161032,200400205720,1ª câmara cível,2016-02-11,"Vistos, relatados e discutidos estes autos, a...",RECURSO EXTRAORDINÁRIO SOBRESTADO NA PRESIDÊN...,JUÍZA CONVOCADA ANA BERNADETE LEITE DE CARVAL...,JUÍZA CONVOCADA ANA BERNADETE LEITE DE CARVAL...
4,des. osório de araújo ramos filho,apelação,201511058,200400210132,1ª câmara cível,2015-07-17,"Vistos, relatados e discutidos os autos da pr...",APELAÇÃO CÍVEL ? AÇÃO REVISIONAL ? CONTRATO B...,"O BANCO DO ESTADO DE SERGIPE S/A - BANESE, in...","O BANCO DO ESTADO DE SERGIPE S/A - BANESE, in..."


Poderíamos também converter os campos `txtAcordao`, `txtEmenta` e `txtRelatório` para caixa baixa e, além disso, remover pontuação, quebras de linhas, etc. No entanto, não queremos eliminar nenhuma possibilidade de análise de forma precipitada. De repente, o pesquisador descobre que caixa-alta dentro dos textos representa algum tipo de cabeçalho (será? não sei) e quer fazer uma análise em cima disso. Portanto, deixemos assim mesmo.

Agora podemos gerar nosso arquivo CSV:

In [11]:
df.to_csv('dataset/asg.csv.gz', index=False)

O arquivo ficou com 638M. Ah! O read_csv do pandas consegue ler arquivos comprimidos, não precisa decompactá-los. Agora podemos aposentar esses arquivos XML. Nos próximos notebooks usaremos somente esse arquivo CSV gerado.

Disponibilizei o arquivo para [download](https://osf.io/4daqk/?action=download).